In [18]:
import os
import sys
import pickle
import psutil
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split

# Define data path
DATA_PATH = "data/"
CHECKPOINT_PATH = "models/"

In [19]:
pids = pickle.load(open(os.path.join(DATA_PATH,'pids.pkl'), 'rb'))
vids = pickle.load(open(os.path.join(DATA_PATH,'vids.pkl'), 'rb'))
targs = pickle.load(open(os.path.join(DATA_PATH,'targets.pkl'), 'rb'))
prob_targs = pickle.load(open(os.path.join(DATA_PATH, 'prob_targets_allvisits.pkl'), 'rb'))
seqs = pickle.load(open(os.path.join(DATA_PATH,'seqs.pkl'), 'rb'))
diags = pickle.load(open(os.path.join(DATA_PATH,'diags.pkl'), 'rb'))
codes = pickle.load(open(os.path.join(DATA_PATH,'icd9.pkl'), 'rb'))
categories = pickle.load(open(os.path.join(DATA_PATH,'categories.pkl'), 'rb'))
sub_categories = pickle.load(open(os.path.join(DATA_PATH,'subcategories.pkl'), 'rb'))
assert len(pids) == len(vids) == len(targs) == len(seqs)

In [20]:
embedding_matrix = torch.load(os.path.join(DATA_PATH, 'embedding_matrix.pt'))

In [21]:
class CustomDataset(Dataset):
    
    def __init__(self, seqs, targets):
        self.x = seqs
        self.y = targets
    
    def __len__(self):
        return(len(self.x))
    
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

In [22]:
# dataset = CustomDataset(seqs, targs)
dataset = CustomDataset(seqs, prob_targs)

In [36]:
def collate_fn(data):
    """
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.long
        x_masks: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.bool
        y: a tensor of shape (# patiens, max # categories) of type torch.long
        y_masks: a tensor of shape (# patiens, max # categories) of type torch.bool
    """
#     sequences, targets = zip(*data)

#     num_patients = len(sequences)
#     num_visits = [len(patient) for patient in sequences]
#     num_codes = [len(visit) for patient in sequences for visit in patient]
#     batch_num_categories = [len(visit) for patient in targets for visit in patient]
#     max_num_visits = max(num_visits)
#     max_num_codes = max(num_codes)
#     max_num_categories = max(batch_num_categories)
    
#     x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
#     x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     y = torch.zeros((num_patients, max_num_categories), dtype=torch.long)
#     y_masks = torch.zeros((num_patients, max_num_categories), dtype=torch.bool)
#     for i_patient, patient in enumerate(sequences):   
#         for j_visit, visit in enumerate(patient[:-1]):
#             for k_code, code in enumerate(visit):
#                 x[i_patient, j_visit, k_code] = code
#                 x_masks[i_patient, j_visit, k_code] = 1

#     for i_patient, patient in enumerate(targets):
#         last_visit = patient[-1]
#         y[i_patient,:len(last_visit)] = torch.LongTensor(last_visit)
#         y_masks[i_patient,:len(last_visit)] = torch.BoolTensor(np.ones(len(last_visit)))
    
#     return x, x_masks, y, y_masks

    sequences, targets = zip(*data)

    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]
    batch_num_categories = [len(visit) for patient in targets for visit in patient]
    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
#     max_num_categories = max(batch_num_categories)
    max_num_categories = len(targets[0][0])
    
    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     y = torch.zeros((num_patients, max_num_categories), dtype=torch.long)
    y = torch.zeros((num_patients, max_num_categories), dtype=torch.float)
    y_masks = torch.zeros((num_patients, max_num_categories), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):   
        for j_visit, visit in enumerate(patient[:-1]):
            for k_code, code in enumerate(visit):
                x[i_patient, j_visit, k_code] = code
                x_masks[i_patient, j_visit, k_code] = 1

#     for i_patient, patient in enumerate(targets):
# #         import pdb; pdb.set_trace()
#         last_visit = patient[-1]
#         y[i_patient,:len(last_visit)] = torch.LongTensor(last_visit)
#         y_masks[i_patient,:len(last_visit)] = torch.BoolTensor(np.ones(len(last_visit)))
    for i_patient, patient in enumerate(targets):
#         import pdb; pdb.set_trace()
        last_visit = patient[-1]
        y[i_patient] = torch.FloatTensor(last_visit)
        y_masks[i_patient] = torch.BoolTensor(np.ones(max_num_categories))
    
    return x, x_masks, y, y_masks

In [37]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)
val_split = int(len(dataset)*0.10)

In [38]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)

lengths = [train_split, test_split, len(dataset) - (train_split + test_split)]
train_dataset, test_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of test dataset:", len(test_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 6561
Length of test dataset: 1312
Length of val dataset: 875


In [39]:
def load_data(train_dataset, test_dataset, val_dataset, collate_fn):
    '''
    Arguments:
        train_dataset: train dataset of type `CustomDataset`
        test_dataset: test dataset of type `CustomDataset`
        val_dataset: validation dataset of type `CustomDataset`
        collate_fn: collate function
        
    Outputs:
        train_loader, test_loader, val_loader: train, test and validation dataloaders
    '''
    batch_size = 100
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               collate_fn=collate_fn,
                                               shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=batch_size,
                                           collate_fn=collate_fn,
                                           shuffle=False)
    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             collate_fn=collate_fn,
                                             shuffle=False)
    
    return train_loader, test_loader, val_loader


train_loader, test_loader, val_loader = load_data(train_dataset, test_dataset, val_dataset, collate_fn)

In [40]:
def get_last_visit(hidden_states, masks):
    """
    Arguments:
        hidden_states: the hidden states of each visit of shape (batch_size, # visits, embedding_dim)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        last_hidden_state: the hidden state for the last true visit of shape (batch_size, embedding_dim)
    """
    idx_vector = masks.any(dim=2).sum(1) - 1
    p_idx = torch.arange(0,len(hidden_states), dtype=torch.int64)
    last_hidden_state = hidden_states[p_idx,idx_vector]
    return last_hidden_state

In [41]:
class EnhancedRNN(nn.Module):
    
    def __init__(self, num_codes, num_categories):
        super().__init__()
        """
        Arguments:
            num_codes: total number of diagnosis codes
            num_categories: total number of higher level categories to predict
        """
        self.embedding = nn.Linear(num_codes, 300)
        self.embedding.weight.data = embedding_matrix
        self.rnn = nn.GRU(300, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(128, num_categories)
    
    def forward(self, x, masks):
        """
        Arguments:
            x: the diagnosis sequence of shape (batch_size, # visits, # diagnosis codes)
            masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

        Outputs:
            logits: logits of shape (batch_size, # categories)
        """
        x = indices_to_multihot(x, masks, 4903)
        x = self.embedding(x)
        x[~masks.any(dim=2)] = 0  # Set masked visits to 0
        output, _ = self.rnn(x)
        true_h_n = get_last_visit(output, masks)
        logits = self.fc(true_h_n)        
        return logits

# load the model here
enhanced_rnn = EnhancedRNN(num_codes = len(codes), num_categories=len(sub_categories))
enhanced_rnn

EnhancedRNN(
  (embedding): Linear(in_features=4903, out_features=300, bias=True)
  (rnn): GRU(300, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=184, bias=True)
)

In [42]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(baseline_rnn.parameters(), lr=0.001)
optimizer = torch.optim.Adadelta(enhanced_rnn.parameters(), weight_decay=0.001)

In [43]:
def eval_model(model, test_loader, k=15, n=-1):
    """
    Arguments:
        model: the EnhancedRNN model
        test_loader: validation dataloader
        k: value for top k predictions
        n: num of records to evaluate in the batch, value -1 evaulates all records
        
    Outputs:
        precision_k: visit-level precison@k
        accuracy_k: code-level accuracy@k
    """
#     y_pred = torch.LongTensor()
#     y_true = torch.LongTensor()
#     all_precision = []
#     all_accuracy = []
    
#     model.eval()
#     with torch.no_grad():
#         for x, masks, y, y_masks in test_loader:
#             n_eval = y.shape[0] - 1 if n == -1 else n
#             y_hat = model(x, masks)
#             y_hat = F.softmax(y_hat, dim=-1)
#             y_multihot = indices_to_multihot(y, y_masks, y_hat.shape[-1])
#             k_correct = 0
#             total_precision = 0
#             total_accuracy = 0
#             for i in range(n_eval):
#                 visit_correct = 0
#                 y_true = y[i, y_masks[i]]
#                 _, y_pred = torch.topk(y_hat[i], k)
                
#                 for v in y_true:
#                     if v in y_pred:
#                         visit_correct += 1
                        
#                 visit_precision = visit_correct / min(k, len(y_true))
#                 visit_accuracy = visit_correct / len(y_true)
#                 k_correct += visit_correct
#                 total_precision += visit_precision
#                 total_accuracy += visit_accuracy

#             precision_k = total_precision / n_eval
#             accuracy_k = total_accuracy / n_eval
#             all_precision.append(precision_k)
#             all_accuracy.append(accuracy_k)

#     total_precision_k = np.mean(all_precision)
#     total_accuracy_k = np.mean(all_accuracy)
#     return total_precision_k, total_accuracy_k

    y_pred = torch.LongTensor()
    y_true = torch.LongTensor()
    all_precision = []
    all_accuracy = []
    
    model.eval()
    with torch.no_grad():
        for x, masks, y, y_masks in test_loader:
            n_eval = y.shape[0] - 1 if n == -1 else n
            y_hat = model(x, masks)
            y_hat = F.softmax(y_hat, dim=-1)
            #y_multihot = indices_to_multihot(y, y_masks, y_hat)
            nz_rows, nz_cols = torch.nonzero(y, as_tuple=True)
            k_correct = 0
            total_precision = 0
            total_accuracy = 0
            for i in range(n_eval):
                visit_correct = 0
                #y_true = y[i, y_masks[i]]
                y_true = nz_cols[nz_rows == i]
                _, y_pred = torch.topk(y_hat[i], k)
                for v in y_true:
                    if v in y_pred:
                        visit_correct += 1
                visit_precision = visit_correct / min(k, len(y_true))
                visit_accuracy = visit_correct / len(y_true)
                k_correct += visit_correct
                total_precision += visit_precision
                total_accuracy += visit_accuracy
            precision_k = total_precision / n_eval
            accuracy_k = total_accuracy / n_eval
            all_precision.append(precision_k)
            all_accuracy.append(accuracy_k)
            
    total_precision_k = np.mean(all_precision)
    total_accuracy_k = np.mean(all_accuracy)
    return total_precision_k, total_accuracy_k

In [44]:
def train(model, train_loader, val_loader, n_epochs):
    """
    Arguments:
        model: the EnhancedRNN model
        train_loader: training dataloder
        val_loader: validation dataloader
        n_epochs: total number of epochs
    """
#     max_cpu, max_ram = print_cpu_usage()
#     for epoch in range(n_epochs):
#         model.train()
#         train_loss = 0
#         for x, x_masks, y, y_masks in train_loader:
#             y_hat = model(x, x_masks)
#             y_mh = indices_to_multihot(y, y_masks, y_hat.shape[-1])
#             loss = criterion(y_hat, y_mh)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             train_loss += loss.item()
#         train_loss = train_loss / len(train_loader)
#         cpu, ram = print_cpu_usage()
#         max_cpu = cpu if cpu > max_cpu else max_cpu
#         max_ram = ram if ram > max_ram else max_ram
#         print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
#         for k in range(5, 31, 5):
#             precision_k, accuracy_k = eval_model(model, val_loader, k=k)
#             print(f'Epoch: {epoch+1} \t Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')
#     final_cpu, final_ram = print_cpu_usage()
#     print(f"Max CPU usage: {max_cpu:.3f}\tMax RAM % usage: {max_ram}")
    
    max_cpu, max_ram = print_cpu_usage()
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, x_masks, y, y_masks in train_loader:
            y_hat = model(x, x_masks)                  
#             y_mh = indices_to_multihot(y, y_masks, y_hat)
            loss = criterion(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        cpu, ram = print_cpu_usage()
        max_cpu = cpu if cpu > max_cpu else max_cpu
        max_ram = ram if ram > max_ram else max_ram
        print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
        for k in range(5, 31, 5):
            precision_k, accuracy_k = eval_model(model, val_loader, k=k)
            print(f'Epoch: {epoch+1} \t Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')
    final_cpu, final_ram = print_cpu_usage()
    print(f"Max CPU usage: {max_cpu:.3f}\tMax RAM % usage: {max_ram}")

In [45]:
def indices_to_multihot(indices, masks, dim):
    mh_size = tuple(indices.shape[:-1]) + (dim,)
    multihot = torch.zeros(mh_size, dtype=torch.float)
    if len(mh_size) == 3:
        for i, patient in enumerate(indices):
            for j, visit in enumerate(patient):
                if masks[i,j].sum() == 0:
                    break
                y_idx = visit[masks[i,j]].unique()
                multihot[i,j] = F.one_hot(y_idx.to(torch.int64), multihot.shape[-1]).sum(0)
    else:
        for idx, row in enumerate(indices):
            y_idx = row[masks[idx]].unique()
            multihot[idx] = F.one_hot(y_idx, dim).sum(0).float()
    return multihot

In [46]:
def print_cpu_usage():
    load = psutil.getloadavg()[2]
    cpu_usage = (load/os.cpu_count()) * 100
    ram = psutil.virtual_memory()[2]
    print(f"CPU: {cpu_usage:0.2f}")
    print(f"RAM %: {ram}")
    return cpu_usage, ram

In [49]:
n_epochs = 100
%time train(enhanced_rnn, train_loader, val_loader, n_epochs)

CPU: 17.28
RAM %: 58.9
CPU: 17.74
RAM %: 58.9
Epoch: 1 	 Training Loss: 3.885364
Epoch: 1 	 Validation precision@k5: 0.4990, accuracy@k5: 0.2691
Epoch: 1 	 Validation precision@k10: 0.5131, accuracy@k10: 0.4415
Epoch: 1 	 Validation precision@k15: 0.5687, accuracy@k15: 0.5524
Epoch: 1 	 Validation precision@k20: 0.6417, accuracy@k20: 0.6395
Epoch: 1 	 Validation precision@k25: 0.7061, accuracy@k25: 0.7059
Epoch: 1 	 Validation precision@k30: 0.7647, accuracy@k30: 0.7647
CPU: 17.64
RAM %: 58.4
Epoch: 2 	 Training Loss: 3.815735
Epoch: 2 	 Validation precision@k5: 0.5424, accuracy@k5: 0.2923
Epoch: 2 	 Validation precision@k10: 0.5172, accuracy@k10: 0.4445
Epoch: 2 	 Validation precision@k15: 0.5807, accuracy@k15: 0.5640
Epoch: 2 	 Validation precision@k20: 0.6574, accuracy@k20: 0.6552
Epoch: 2 	 Validation precision@k25: 0.7210, accuracy@k25: 0.7208
Epoch: 2 	 Validation precision@k30: 0.7735, accuracy@k30: 0.7735
CPU: 17.70
RAM %: 58.3
Epoch: 3 	 Training Loss: 3.781381
Epoch: 3 	 Vali

CPU: 17.72
RAM %: 58.7
Epoch: 19 	 Training Loss: 3.577937
Epoch: 19 	 Validation precision@k5: 0.6311, accuracy@k5: 0.3454
Epoch: 19 	 Validation precision@k10: 0.5921, accuracy@k10: 0.5128
Epoch: 19 	 Validation precision@k15: 0.6327, accuracy@k15: 0.6150
Epoch: 19 	 Validation precision@k20: 0.6980, accuracy@k20: 0.6957
Epoch: 19 	 Validation precision@k25: 0.7602, accuracy@k25: 0.7600
Epoch: 19 	 Validation precision@k30: 0.8090, accuracy@k30: 0.8090
CPU: 17.70
RAM %: 58.8
Epoch: 20 	 Training Loss: 3.572452
Epoch: 20 	 Validation precision@k5: 0.6233, accuracy@k5: 0.3403
Epoch: 20 	 Validation precision@k10: 0.5970, accuracy@k10: 0.5151
Epoch: 20 	 Validation precision@k15: 0.6488, accuracy@k15: 0.6307
Epoch: 20 	 Validation precision@k20: 0.7118, accuracy@k20: 0.7093
Epoch: 20 	 Validation precision@k25: 0.7710, accuracy@k25: 0.7708
Epoch: 20 	 Validation precision@k30: 0.8163, accuracy@k30: 0.8163
CPU: 17.82
RAM %: 58.6
Epoch: 21 	 Training Loss: 3.564807
Epoch: 21 	 Validation 

Epoch: 36 	 Validation precision@k30: 0.8253, accuracy@k30: 0.8253
CPU: 18.07
RAM %: 58.7
Epoch: 37 	 Training Loss: 3.476659
Epoch: 37 	 Validation precision@k5: 0.6712, accuracy@k5: 0.3696
Epoch: 37 	 Validation precision@k10: 0.6337, accuracy@k10: 0.5468
Epoch: 37 	 Validation precision@k15: 0.6721, accuracy@k15: 0.6533
Epoch: 37 	 Validation precision@k20: 0.7286, accuracy@k20: 0.7262
Epoch: 37 	 Validation precision@k25: 0.7856, accuracy@k25: 0.7854
Epoch: 37 	 Validation precision@k30: 0.8305, accuracy@k30: 0.8305
CPU: 17.97
RAM %: 58.6
Epoch: 38 	 Training Loss: 3.473370
Epoch: 38 	 Validation precision@k5: 0.6679, accuracy@k5: 0.3672
Epoch: 38 	 Validation precision@k10: 0.6237, accuracy@k10: 0.5384
Epoch: 38 	 Validation precision@k15: 0.6626, accuracy@k15: 0.6439
Epoch: 38 	 Validation precision@k20: 0.7253, accuracy@k20: 0.7228
Epoch: 38 	 Validation precision@k25: 0.7841, accuracy@k25: 0.7839
Epoch: 38 	 Validation precision@k30: 0.8280, accuracy@k30: 0.8280
CPU: 17.79
RAM 

Epoch: 54 	 Validation precision@k25: 0.7989, accuracy@k25: 0.7986
Epoch: 54 	 Validation precision@k30: 0.8412, accuracy@k30: 0.8412
CPU: 18.59
RAM %: 58.6
Epoch: 55 	 Training Loss: 3.389217
Epoch: 55 	 Validation precision@k5: 0.6953, accuracy@k5: 0.3812
Epoch: 55 	 Validation precision@k10: 0.6514, accuracy@k10: 0.5622
Epoch: 55 	 Validation precision@k15: 0.6896, accuracy@k15: 0.6700
Epoch: 55 	 Validation precision@k20: 0.7450, accuracy@k20: 0.7423
Epoch: 55 	 Validation precision@k25: 0.8006, accuracy@k25: 0.8004
Epoch: 55 	 Validation precision@k30: 0.8406, accuracy@k30: 0.8406
CPU: 18.66
RAM %: 58.6
Epoch: 56 	 Training Loss: 3.383726
Epoch: 56 	 Validation precision@k5: 0.6976, accuracy@k5: 0.3819
Epoch: 56 	 Validation precision@k10: 0.6476, accuracy@k10: 0.5593
Epoch: 56 	 Validation precision@k15: 0.6834, accuracy@k15: 0.6640
Epoch: 56 	 Validation precision@k20: 0.7418, accuracy@k20: 0.7392
Epoch: 56 	 Validation precision@k25: 0.7935, accuracy@k25: 0.7932
Epoch: 56 	 Val

Epoch: 72 	 Validation precision@k20: 0.7449, accuracy@k20: 0.7423
Epoch: 72 	 Validation precision@k25: 0.8005, accuracy@k25: 0.8003
Epoch: 72 	 Validation precision@k30: 0.8388, accuracy@k30: 0.8388
CPU: 18.89
RAM %: 58.7
Epoch: 73 	 Training Loss: 3.336516
Epoch: 73 	 Validation precision@k5: 0.6967, accuracy@k5: 0.3814
Epoch: 73 	 Validation precision@k10: 0.6540, accuracy@k10: 0.5640
Epoch: 73 	 Validation precision@k15: 0.6895, accuracy@k15: 0.6701
Epoch: 73 	 Validation precision@k20: 0.7451, accuracy@k20: 0.7426
Epoch: 73 	 Validation precision@k25: 0.7978, accuracy@k25: 0.7976
Epoch: 73 	 Validation precision@k30: 0.8379, accuracy@k30: 0.8379
CPU: 18.73
RAM %: 58.7
Epoch: 74 	 Training Loss: 3.333738
Epoch: 74 	 Validation precision@k5: 0.6989, accuracy@k5: 0.3812
Epoch: 74 	 Validation precision@k10: 0.6530, accuracy@k10: 0.5624
Epoch: 74 	 Validation precision@k15: 0.6862, accuracy@k15: 0.6667
Epoch: 74 	 Validation precision@k20: 0.7424, accuracy@k20: 0.7399
Epoch: 74 	 Val

Epoch: 90 	 Validation precision@k15: 0.6850, accuracy@k15: 0.6656
Epoch: 90 	 Validation precision@k20: 0.7431, accuracy@k20: 0.7405
Epoch: 90 	 Validation precision@k25: 0.7948, accuracy@k25: 0.7946
Epoch: 90 	 Validation precision@k30: 0.8398, accuracy@k30: 0.8398
CPU: 21.70
RAM %: 56.4
Epoch: 91 	 Training Loss: 3.308068
Epoch: 91 	 Validation precision@k5: 0.7023, accuracy@k5: 0.3832
Epoch: 91 	 Validation precision@k10: 0.6554, accuracy@k10: 0.5652
Epoch: 91 	 Validation precision@k15: 0.6912, accuracy@k15: 0.6718
Epoch: 91 	 Validation precision@k20: 0.7445, accuracy@k20: 0.7420
Epoch: 91 	 Validation precision@k25: 0.8018, accuracy@k25: 0.8016
Epoch: 91 	 Validation precision@k30: 0.8411, accuracy@k30: 0.8411
CPU: 21.70
RAM %: 56.5
Epoch: 92 	 Training Loss: 3.306335
Epoch: 92 	 Validation precision@k5: 0.6940, accuracy@k5: 0.3782
Epoch: 92 	 Validation precision@k10: 0.6527, accuracy@k10: 0.5623
Epoch: 92 	 Validation precision@k15: 0.6917, accuracy@k15: 0.6721
Epoch: 92 	 Val

In [50]:
for k in range(5, 31, 5):
    precision_k, accuracy_k = eval_model(enhanced_rnn, test_loader, k=k)
    print(f'Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')

Validation precision@k5: 0.7111, accuracy@k5: 0.3860
Validation precision@k10: 0.6585, accuracy@k10: 0.5675
Validation precision@k15: 0.6932, accuracy@k15: 0.6732
Validation precision@k20: 0.7484, accuracy@k20: 0.7455
Validation precision@k25: 0.7959, accuracy@k25: 0.7958
Validation precision@k30: 0.8348, accuracy@k30: 0.8348


In [51]:
torch.save(enhanced_rnn, os.path.join(CHECKPOINT_PATH, "EnhancedRNN_100.pth"))